In [1]:
from typing import Dict, Optional, Tuple
from overrides import overrides

import os
import copy
import torch
import numpy as np
from pathlib import Path
from scripts.to_npy import ToNpy

from allennlp.nn import util
from allennlp.data import Vocabulary
from allennlp.models.model import Model
from allennlp.modules.token_embedders import Embedding
from allennlp.modules import FeedForward, TextFieldEmbedder, Seq2VecEncoder
from allennlp.nn import InitializerApplicator, RegularizerApplicator
from allennlp.training.metrics.categorical_accuracy import CategoricalAccuracy
from allennlp.nn.activations import Activation
from allennlp.predictors.predictor import Predictor

from table_embedder.models.lib.bert_token_embedder import PretrainedBertEmbedder

from table_embedder.models.embedder_util import TableUtil
# from embedder_util import TableUtil
# from embedder_util import PredUtil
from table_embedder.models.lib.stacked_self_attention import StackedSelfAttentionEncoder
# from lib.stacked_self_attention import StackedSelfAttentionEncoder
from allennlp.models.archival import load_archive

from table_embedder.models.cache_util import CacheUtil

from torch import nn
from torch import tensor
import pdb

/work/arnaik_umass_edu/.conda/envs/table_emb_py37/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/work/arnaik_umass_edu/.conda/envs/table_emb_py37/lib/python3.7/site-packages/apex/pyprof/__init__.py:5: FutureWarning: pyprof will be removed by the end of June, 2022
  warnings.warn("pyprof will be removed by the end of June, 2022", FutureWarning)


In [7]:
from torch import tensor

In [10]:
#PretrainedBertEmbedder(pretrained_model='bert-base-uncased', top_layer_only=True)

In [11]:
table_info = [{'table_id': '1438042981460.12__CC-MAIN-20150728002301-00003-ip-10-236-191-2.ec2.internal__1387'
               , 'num_rows': 3
               , 'num_cols': 5
               , 'header': ['size', 'us', 'inches', 'china', 'japan (cm)']
               , 'cell_labels': None
               , 'col_labels': None
               , 'table_labels': None
                , 'table_data_raw': [['6/7', 'xs', '9.125', '235','23']
                                     , ['s', '7/8', '40-41', '245','24']
                                     , ['m', '8/9', '9.875', '255','25']]
                , 'table': [['size', 'us', 'inches', 'china', 'japan (cm)']
                            , ['6/7', 'xs', '9.125', '235','23']
                            , ['s', '7/8', '40-41', '245','24']
                            , ['m', '8/9', '9.875', '255','25']]}]

In [12]:
indexed_headers = {'bert': {'token_ids': tensor([[[ 101, 2946,  102,    0,    0,    0],
         [ 101, 2149,  102,    0,    0,    0],
         [ 101, 5282,  102,    0,    0,    0],
         [ 101, 2859,  102,    0,    0,    0],
        [ 101, 2900, 1006, 4642, 1007,  102]]], device='cuda:0')
, 'mask': tensor([[[ True,  True,  True, False, False, False],
         [ True,  True,  True, False, False, False],
         [ True,  True,  True, False, False, False],
         [ True,  True,  True, False, False, False],
        [ True,  True,  True,  True,  True,  True]]], device='cuda:0')
, 'type_ids': tensor([[[0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0]]], device='cuda:0')}}

In [13]:
indexed_cells = {'bert': {'token_ids': tensor([[[[  101,  1020,  1013,  1021,   102],
          [  101,  1060,  2015,   102,     0],
          [  101,  1023,  1012,  8732,   102],
          [  101, 17825,   102,     0,   0],[101,  2603,   102,     0,     0]],

         [[  101,  1055,   102,     0,     0],
          [  101,  1021,  1013,  1022,   102],
          [  101,  2871,  1011,  4601,   102],
          [  101, 21005,   102,     0,     0],
           [101,  2484,   102,     0,     0]],

         [[  101,  1049,   102,     0,     0],
          [  101,  1022,  1013,  1023,   102],
          [  101,  1023,  1012, 27658,   102],
          [  101, 20637,   102,     0,     0],
          [ 101,  2423,   102,     0,     0]]]], device='cuda:0')
, 'mask': tensor([[[[ True,  True,  True,  True,  True],
          [ True,  True,  True,  True, False],
          [ True,  True,  True,  True,  True],
          [ True,  True,  True, False, False],
        [ True,  True,  True, False, False]],

         [[ True,  True,  True, False, False],
          [ True,  True,  True,  True,  True],
          [ True,  True,  True,  True,  True],
          [ True,  True,  True, False, False],
         [ True,  True,  True, False, False]],

         [[ True,  True,  True, False, False],
          [ True,  True,  True,  True,  True],
          [ True,  True,  True,  True,  True],
          [ True,  True,  True, False, False],
          [ True,  True,  True, False, False]]]], device='cuda:0')
, 'type_ids': tensor([[[[0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0]],

         [[0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0]],

         [[0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0]]]], device='cuda:0')}}

In [5]:
type(table_info), type(indexed_headers), type(indexed_cells)

(list, dict, dict)

In [10]:
model = TableEmbedderTorch().to('cuda:0')

In [14]:
loss, prob = model(table_info,indexed_headers, indexed_cells, None)

In [15]:
prob.shape

torch.Size([1, 127656])

In [9]:
class TableEmbedderTorch(nn.Module):

    def __init__(self) -> None:
        super(TableEmbedderTorch,self).__init__()

        self.num_max_row_pos = 31  # FIXME: why different than the row_pos_embedding in cp.jsonnet? -- Corrected it to be 31 as per the jsonnet. The model is saved accordingly
        self.num_max_col_pos = 25
        self.n_classes = 127656  # FIXME: we'll update this, since I can't reproduce
#        self.n_seed_cols = int(os.getenv('n_seed_cols'))
        ## As per the paper n_seed_cols = 1/2/3
        self.n_seed_cols = 5
        """
        from exp/col_pop/cp.jsonnet
        "row_pos_embedding": {
            "num_embeddings": 31,
            "embedding_dim": 768
        },
        "col_pos_embedding": {
            "num_embeddings": 25,
            "embedding_dim": 768
        },
        """
        self.row_pos_embedding = Embedding(num_embeddings=self.num_max_row_pos, embedding_dim=768)
        self.col_pos_embedding = Embedding(num_embeddings=self.num_max_col_pos, embedding_dim=768)

        """
        "top_feedforward": {
          "input_dim": 768*3,
          # "input_dim": 768,
          "num_layers": 1,
          "hidden_dims": [127656],
          "activations": ["linear"],
          "dropout": [0.0]
        }
        """
        # FIXME: replace with normal torch.nn.Linear
        # NOTE: this is the only parameter not initialized from the pretrained TABBIE model
        # NOTE : Activation used in linear i.e. basically as per allennlp module it is ignored - https://github.com/allenai/allennlp/blob/master/allennlp/nn/activations.py#L78
        
        self.top_feedforward = nn.Linear(768*self.n_seed_cols, self.n_classes)
        self.Dropout = nn.Dropout(p = 0.0)
#        self.top_feedforward = FeedForward(768*3, 1, [self.n_classes], Activation.by_name("linear"), [0.0])

        """
        "bert_embbeder": {
            "pretrained_model": "bert-base-uncased",
            "top_layer_only": true,
        }
        """
        self.bert_embedder = PretrainedBertEmbedder(pretrained_model='bert-base-uncased', top_layer_only=True)

        """
        from exp/col_pop/cp.jsonnet
        "transformer_row1": {
              "input_dim": 768,
              "hidden_dim": 768,
              "projection_dim": 768,
              "feedforward_hidden_dim": 3072,
              # "dropout_prob": 0.0,
              # "residual_dropout_prob": 0.0,
              # "attention_dropout_prob": 0.0,
              "num_layers": 1,
              "num_attention_heads": 12,
              "use_positional_encoding": false,
            }
        """
        self.transformer_col1 = StackedSelfAttentionEncoder(768, 768, 768, 3072, 1, 12, False)
        self.transformer_col2 = StackedSelfAttentionEncoder(768, 768, 768, 3072, 1, 12, False)
        self.transformer_col3 = StackedSelfAttentionEncoder(768, 768, 768, 3072, 1, 12, False)
        self.transformer_col4 = StackedSelfAttentionEncoder(768, 768, 768, 3072, 1, 12, False)
        self.transformer_col5 = StackedSelfAttentionEncoder(768, 768, 768, 3072, 1, 12, False)
        self.transformer_col6 = StackedSelfAttentionEncoder(768, 768, 768, 3072, 1, 12, False)
        self.transformer_col7 = StackedSelfAttentionEncoder(768, 768, 768, 3072, 1, 12, False)
        self.transformer_col8 = StackedSelfAttentionEncoder(768, 768, 768, 3072, 1, 12, False)
        self.transformer_col9 = StackedSelfAttentionEncoder(768, 768, 768, 3072, 1, 12, False)
        self.transformer_col10 = StackedSelfAttentionEncoder(768, 768, 768, 3072, 1, 12, False)
        self.transformer_col11 = StackedSelfAttentionEncoder(768, 768, 768, 3072, 1, 12, False)
        self.transformer_col12 = StackedSelfAttentionEncoder(768, 768, 768, 3072, 1, 12, False)

        self.transformer_row1 = StackedSelfAttentionEncoder(768, 768, 768, 3072, 1, 12, False)
        self.transformer_row2 = StackedSelfAttentionEncoder(768, 768, 768, 3072, 1, 12, False)
        self.transformer_row3 = StackedSelfAttentionEncoder(768, 768, 768, 3072, 1, 12, False)
        self.transformer_row4 = StackedSelfAttentionEncoder(768, 768, 768, 3072, 1, 12, False)
        self.transformer_row5 = StackedSelfAttentionEncoder(768, 768, 768, 3072, 1, 12, False)
        self.transformer_row6 = StackedSelfAttentionEncoder(768, 768, 768, 3072, 1, 12, False)
        self.transformer_row7 = StackedSelfAttentionEncoder(768, 768, 768, 3072, 1, 12, False)
        self.transformer_row8 = StackedSelfAttentionEncoder(768, 768, 768, 3072, 1, 12, False)
        self.transformer_row9 = StackedSelfAttentionEncoder(768, 768, 768, 3072, 1, 12, False)
        self.transformer_row10 = StackedSelfAttentionEncoder(768, 768, 768, 3072, 1, 12, False)
        self.transformer_row11 = StackedSelfAttentionEncoder(768, 768, 768, 3072, 1, 12, False)
        self.transformer_row12 = StackedSelfAttentionEncoder(768, 768, 768, 3072, 1, 12, False)

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # FIXME: we'll try loss both ways
        self.loss = torch.nn.BCELoss()
        # self.loss_func = torch.nn.CrossEntropyLoss()

        # FIXME: unused, torch.load_state_dict with strict false
        # self.compose_ff = compose_ff
        # self.feedforward = feedforward

        self.cache_usage = os.getenv("cache_usage")
        # FIXME: maybe don't pull so much from env variables
        # TODO: try these as plain parameters, initialized from the saved clscol and clsrow embeddings
        self.cls_col = np.load('/work/arnaik_umass_edu/tabbie/data/clscol.npy')
        self.cls_row = np.load('/work/arnaik_umass_edu/tabbie/data/clsrow.npy')

        self.opt_level = 'O0'

        if self.cache_usage is not None:
            self.cache_util = CacheUtil(self.cache_usage, os.getenv("cell_db_path"))
        else:
            self.cache_util = None
            
        self.init_weight()

    def init_weight(self):
        model_parameters = dict(self.named_parameters())
        archived_parameters = torch.load('/work/arnaik_umass_edu/model_named.pt') # Archieved weights of feedforward layers discarded.
        for name, weights in archived_parameters.items():
            if name in model_parameters:
                new_weights = weights.data
                model_parameters[name].data.copy_(new_weights)

    def get_tabemb(self, bert_header, bert_data, n_rows, n_cols, bs, table_mask, nrows, ncols):
        row_pos_ids = torch.arange(0, self.num_max_row_pos, device=self.device, dtype=torch.long)
        col_pos_ids = torch.arange(0, self.num_max_col_pos, device=self.device, dtype=torch.long)

        n_rows += 1  # row CLS
        n_cols += 1  # col CLS
        # NOTE: there is no need for a copy, but it doesn't really matter
        cls_col = torch.from_numpy(copy.deepcopy(self.cls_col)).to(device=self.device)
        cls_row = torch.from_numpy(copy.deepcopy(self.cls_row)).to(device=self.device)
        row_pos_embs = self.row_pos_embedding(row_pos_ids[:n_rows + 1])
        col_pos_embs = self.col_pos_embedding(col_pos_ids[:n_cols])

        for i in range(1, 13):
            transformer_row = getattr(self, 'transformer_row{}'.format(str(i)))
            transformer_col = getattr(self, 'transformer_col{}'.format(str(i)))
            if i == 1:
                bert_data = TableUtil.add_cls_tokens(bert_header, bert_data, cls_row, cls_col, bs, n_rows, n_cols)
                bert_data += row_pos_embs.expand((bs, n_cols, n_rows + 1, 768)).permute(0, 2, 1, 3).expand_as(bert_data)
                bert_data += col_pos_embs.expand((bs, n_rows + 1, n_cols, 768)).expand_as(bert_data)
                # table_mask = TableUtil.add_cls_mask(table_mask, table_info, bs, n_rows, n_cols, self.device)
                table_mask = TableUtil.add_cls_mask(table_mask, bs, n_rows, n_cols, self.device, nrows, ncols)
                col_embs = TableUtil.get_col_embs(bert_data, bs, n_rows, n_cols, table_mask, transformer_col,
                                                  self.opt_level)
                row_embs = TableUtil.get_row_embs(bert_data, bs, n_rows, n_cols, table_mask, transformer_row,
                                                  self.opt_level)
            else:
                row_embs = TableUtil.get_row_embs(ave_embs, bs, n_rows, n_cols, table_mask, transformer_row,
                                                  self.opt_level)
                col_embs = TableUtil.get_col_embs(ave_embs, bs, n_rows, n_cols, table_mask, transformer_col,
                                                  self.opt_level)
            ave_embs = (row_embs + col_embs) / 2.0
        return row_embs, col_embs, n_rows, n_cols

    @staticmethod
    def get_cat_cls(row_embs, col_embs, n_seed_cols, device):
        # Not just colcls concatenated but their average with row_embs concatenated
        ave_embs = (row_embs + col_embs) / 2.0
        ave_embs = ave_embs[:, 0, 1:, :] # [bs, row having clscol, first_row:all_rows, emb_dim]
        bs = ave_embs.shape[0]

        cls_embs = ave_embs[:, 0, :]       
        for i in range(1, n_seed_cols):
            if ave_embs.shape[1] <= i: ## This condition is never met as num clscol embeddings == num_seed_cols
                zeros = torch.zeros((bs, 768), device=device)
                cls_embs = torch.cat([cls_embs, zeros], dim=1)
            else:
                cls_embs = torch.cat([cls_embs, ave_embs[:, i, :]], dim=1)
        return cls_embs

    @staticmethod
    def mask_cls_embs(cls_embs, table_info):
        ## Function to mask cls embedding based on number of columns / number of seed columns present in table
        for k, one_info in enumerate(table_info):
            if one_info['num_cols'] == 1:
                cls_embs[k, 768:] = 0
            elif one_info['num_cols'] == 2:
                cls_embs[k, (768 * 2):] = 0
        return cls_embs

    def validate_seed_cols(self, table_info):
        for one_info in table_info:
            if one_info['num_cols'] > self.n_seed_cols:
                raise ValueError('invalid num cols')

    def get_meta(self, table_info):
        nrows = [one_info['num_rows'] for one_info in table_info]
        ncols = [one_info['num_cols'] for one_info in table_info]
        tids = [one_info['table_id'] for one_info in table_info]
        return nrows, ncols, tids

    @overrides
    def forward(self,
                table_info: Dict[str, str],
                indexed_headers: Dict[str, torch.LongTensor],
                indexed_cells: Dict[str, torch.LongTensor],
                labels: Optional[torch.LongTensor]) -> Tuple[Optional[torch.Tensor], torch.Tensor]:
        """

        :param table_info: see data/table_info.txt for example
        :param indexed_headers:  see data/indexed_headers.txt for example
        :param labels: vector of length self.n_classes 0 or 1
        :return:
        """
        ## Initialize
        self.bert_embedder.eval()
        ## TODO : n_seed_col variable not defined in jsonnets loaded.
        ## Question : What exactly is n_seed_cols?
        self.validate_seed_cols(table_info)
        bs, n_rows, n_cols = TableUtil.get_max_row_col(table_info)
        nrows, ncols, tids = self.get_meta(table_info)
        table_mask = TableUtil.get_table_mask(table_info, bs, n_rows, n_cols, self.device)

        ## Prediction Probability
        ## TODO : indexed_cells not sent for column population.
        ## Question : Why is indexed_cells None for column population?
        bert_header, bert_cell = TableUtil.get_bert_emb(indexed_headers, indexed_cells, table_info, bs, n_rows, n_cols,
                                                        self.cache_usage, self.bert_embedder, self.cache_util,
                                                        self.device)

        row_embs, col_embs, n_rows_cls, n_cols_cls = self.get_tabemb(bert_header, bert_cell, n_rows, n_cols, bs,
                                                                     table_mask, nrows, ncols) # row_embs.shape : torch.Size([1, 6, 5, 768])
        
        ## TODO : Updated n_seed_cols before updating this
        # To fine-tune TABBIE on this task, we first concatenate the column [CLSCOL] 
        # embeddings of the seed table into a single vector and pass it through a single linear 
        # and softmax layer, training with a multi-label classification objective
        
        cls_embs = self.get_cat_cls(row_embs, col_embs, self.n_seed_cols, self.device) # Concatenate clscol embeddings for seed columns     
        cls_embs = self.mask_cls_embs(cls_embs, table_info)
        out_prob = self.top_feedforward(cls_embs)

        # from this point we have the probabilities for each possible header, we'll do BCE loss (or their original duplicating loss)
        if labels is not None:
            ## TODO: compute loss            
            loss = self.loss(out_prob, labels)
        else:
            loss = None

        return loss, out_prob

In [1]:
from torch import tensor
import pdb

/work/arnaik_umass_edu/.conda/envs/table_emb_py37/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
table_info = [{'table_id': '1438042981460.12__CC-MAIN-20150728002301-00003-ip-10-236-191-2.ec2.internal__1387'
                  , 'num_rows': 3
                  , 'num_cols': 2
                  , 'header': ['size', 'japan (cm)']
                  , 'cell_labels': None
                  , 'col_labels': None
                  , 'table_labels': None
                  , 'table_data_raw': [['6/7', '23']
        , ['s', '24']
        , ['m', '25']]
                  , 'table': [['size', 'japan (cm)']
        , ['6/7', '23']
        , ['s', '24']
        , ['m', '25']]},{'table_id': '1438042981460.12__CC-MAIN-20150728002301-00003-ip-10-236-191-2.ec2.internal__1387'
                  , 'num_rows': 3
                  , 'num_cols': 2
                  , 'header': ['size', 'japan (cm)']
                  , 'cell_labels': None
                  , 'col_labels': None
                  , 'table_labels': None
                  , 'table_data_raw': [['6/7', '23']
        , ['s', '24']
        , ['m', '25']]
                  , 'table': [['size', 'japan (cm)']
        , ['6/7', '23']
        , ['s', '24']
        , ['m', '25']]},{'table_id': '1438042981460.12__CC-MAIN-20150728002301-00003-ip-10-236-191-2.ec2.internal__1387'
                  , 'num_rows': 3
                  , 'num_cols': 2
                  , 'header': ['size', 'japan (cm)']
                  , 'cell_labels': None
                  , 'col_labels': None
                  , 'table_labels': None
                  , 'table_data_raw': [['6/7', '23']
        , ['s', '24']
        , ['m', '25']]
                  , 'table': [['size', 'japan (cm)']
        , ['6/7', '23']
        , ['s', '24']
        , ['m', '25']]}]

indexed_headers = {'bert': {'token_ids': tensor([[[101, 2946, 102, 0, 0, 0],
                                                  [101, 2900, 1006, 4642, 1007, 102]]
                                                 ,[[101, 2946, 102, 0, 0, 0],
                                                  [101, 2900, 1006, 4642, 1007, 102]]
                                                 ,[[101, 2946, 102, 0, 0, 0],
                                                  [101, 2900, 1006, 4642, 1007, 102]]], device='cuda:0')
    , 'mask': tensor([[[True, True, True, False, False, False],
                       [True, True, True, True, True, True]],
                     [[True, True, True, False, False, False],
                       [True, True, True, True, True, True]],
                     [[True, True, True, False, False, False],
                       [True, True, True, True, True, True]]], device='cuda:0')
    , 'type_ids': tensor([[[0, 0, 0, 0, 0, 0],
                           [0, 0, 0, 0, 0, 0]],
                         [[0, 0, 0, 0, 0, 0],
                           [0, 0, 0, 0, 0, 0]],
                         [[0, 0, 0, 0, 0, 0],
                           [0, 0, 0, 0, 0, 0]]], device='cuda:0')}}

indexed_cells = {'bert': {'token_ids': tensor([[[[101, 1020, 1013, 1021, 102],[101, 2603, 102, 0, 0]],

                                                [[101, 1055, 102, 0, 0],
                                                 [101, 2484, 102, 0, 0]],

                                                [[101, 1049, 102, 0, 0],
                                                 [101, 2423, 102, 0, 0]]],
                                              [[[101, 1020, 1013, 1021, 102],[101, 2603, 102, 0, 0]],

                                                [[101, 1055, 102, 0, 0],
                                                 [101, 2484, 102, 0, 0]],

                                                [[101, 1049, 102, 0, 0],
                                                 [101, 2423, 102, 0, 0]]],
                                              [[[101, 1020, 1013, 1021, 102],[101, 2603, 102, 0, 0]],

                                                [[101, 1055, 102, 0, 0],
                                                 [101, 2484, 102, 0, 0]],

                                                [[101, 1049, 102, 0, 0],
                                                 [101, 2423, 102, 0, 0]]]], device='cuda:0')
    , 'mask': tensor([[[[True, True, True, True, True],
                        [True, True, True, False, False]],

                       [[True, True, True, False, False],
                        [True, True, True, False, False]],

                       [[True, True, True, False, False],
                        [True, True, True, False, False]]],
                     [[[True, True, True, True, True],
                        [True, True, True, False, False]],

                       [[True, True, True, False, False],
                        [True, True, True, False, False]],

                       [[True, True, True, False, False],
                        [True, True, True, False, False]]],
                     [[[True, True, True, True, True],
                        [True, True, True, False, False]],

                       [[True, True, True, False, False],
                        [True, True, True, False, False]],

                       [[True, True, True, False, False],
                        [True, True, True, False, False]]]], device='cuda:0')
    , 'type_ids': tensor([[[[0, 0, 0, 0, 0],
                            [0, 0, 0, 0, 0]],

                           [[0, 0, 0, 0, 0],
                            [0, 0, 0, 0, 0]],

                           [[0, 0, 0, 0, 0],
                            [0, 0, 0, 0, 0]]],
                         [[[0, 0, 0, 0, 0],
                            [0, 0, 0, 0, 0]],

                           [[0, 0, 0, 0, 0],
                            [0, 0, 0, 0, 0]],

                           [[0, 0, 0, 0, 0],
                            [0, 0, 0, 0, 0]]],
                         [[[0, 0, 0, 0, 0],
                            [0, 0, 0, 0, 0]],

                           [[0, 0, 0, 0, 0],
                            [0, 0, 0, 0, 0]],

                           [[0, 0, 0, 0, 0],
                            [0, 0, 0, 0, 0]]]], device='cuda:0')}}

In [20]:
## Sample Input
table_info = [{'table_id': 'table-0875-744', 'num_rows': 57, 'num_cols': 2, 'header': ['Edition', 'Year'], 'cell_lables': None, 'col_lables': None, 'table_lables': None, 'table_data_raw': [['1st', '1957'], ['2nd', '1958'], ['3rd', '1959'], ['4th', '1960'], ['5th', '1961'], ['6th', '1962'], ['7th', '1963'], ['8th', '1964'], ['9th', '1965'], ['10th', '1966'], ['11th', '1967'], ['12th', '1968'], ['13th', '1969'], ['14th', '1970'], ['15th', '1971'], ['16th', '1972'], ['17th', '1973'], ['18th', '1974'], ['19th', '1975'], ['20th', '1976'], ['21st', '1977'], ['22nd', '1978'], ['23rd', '1979'], ['24th', '1980'], ['25th', '1981'], ['26th', '1982'], ['27th', '1983'], ['28th', '1984'], ['29th', '1985'], ['30th', '1986'], ['31st', '1987'], ['32nd', '1988'], ['33rd', '1989'], ['34th', '1990'], ['35th', '1991'], ['36th', '1992'], ['37th', '1993'], ['38th', '1994'], ['39th', '1995'], ['40th', '1996'], ['41st', '1997'], ['42nd', '1998'], ['43rd', '1999'], ['44th', '2000'], ['45th', '2001'], ['46th', '2002'], ['47th', '2003'], ['48th', '2004'], ['49th', '2005'], ['50th', '2006'], ['—', '2007'], ['51st', '2008'], ['52nd', '2009'], ['53rd', '2010'], ['54th', '2011'], ['55th', '2012'], ['56th', '2013']], 'table': [['Edition', 'Year'], ['1st', '1957'], ['2nd', '1958'], ['3rd', '1959'], ['4th', '1960'], ['5th', '1961'], ['6th', '1962'], ['7th', '1963'], ['8th', '1964'], ['9th', '1965'], ['10th', '1966'], ['11th', '1967'], ['12th', '1968'], ['13th', '1969'], ['14th', '1970'], ['15th', '1971'], ['16th', '1972'], ['17th', '1973'], ['18th', '1974'], ['19th', '1975'], ['20th', '1976'], ['21st', '1977'], ['22nd', '1978'], ['23rd', '1979'], ['24th', '1980'], ['25th', '1981'], ['26th', '1982'], ['27th', '1983'], ['28th', '1984'], ['29th', '1985'], ['30th', '1986'], ['31st', '1987'], ['32nd', '1988'], ['33rd', '1989'], ['34th', '1990'], ['35th', '1991'], ['36th', '1992'], ['37th', '1993'], ['38th', '1994'], ['39th', '1995'], ['40th', '1996'], ['41st', '1997'], ['42nd', '1998'], ['43rd', '1999'], ['44th', '2000'], ['45th', '2001'], ['46th', '2002'], ['47th', '2003'], ['48th', '2004'], ['49th', '2005'], ['50th', '2006'], ['—', '2007'], ['51st', '2008'], ['52nd', '2009'], ['53rd', '2010'], ['54th', '2011'], ['55th', '2012'], ['56th', '2013']]}]

indexed_headers = {'bert': {'token_ids': tensor([[[3179],
         [2095]]], device='cuda:0'), 'mask': tensor([[[True],
         [True]]], device='cuda:0'), 'type_ids': tensor([[[0],
         [0]]], device='cuda:0')}}

indexed_cells = {'bert': {'token_ids': tensor([[[[ 3083,     0],
          [ 3890,     0]],

         [[ 3416,     0],
          [ 3845,     0]],

         [[ 3822,     0],
          [ 3851,     0]],

         [[ 4343,     0],
          [ 3624,     0]],

         [[ 4833,     0],
          [ 3777,     0]],

         [[ 5351,     0],
          [ 3705,     0]],

         [[ 5504,     0],
          [ 3699,     0]],

         [[ 5893,     0],
          [ 3546,     0]],

         [[ 6280,     0],
          [ 3551,     0]],

         [[ 6049,     0],
          [ 3547,     0]],

         [[ 6252,     0],
          [ 3476,     0]],

         [[ 5940,     0],
          [ 3380,     0]],

         [[ 6122,     0],
          [ 3440,     0]],

         [[ 6400,     0],
          [ 3359,     0]],

         [[ 6286,     0],
          [ 3411,     0]],

         [[ 5767,     0],
          [ 3285,     0]],

         [[ 5550,     0],
          [ 3381,     0]],

         [[ 4985,     0],
          [ 3326,     0]],

         [[ 3708,     0],
          [ 3339,     0]],

         [[ 3983,     0],
          [ 3299,     0]],

         [[ 7398,     0],
          [ 3355,     0]],

         [[13816,     0],
          [ 3301,     0]],

         [[13928,     0],
          [ 3245,     0]],

         [[13386,     0],
          [ 3150,     0]],

         [[10965,     0],
          [ 3261,     0]],

         [[14935,     0],
          [ 3196,     0]],

         [[15045,     0],
          [ 3172,     0]],

         [[15538,     0],
          [ 3118,     0]],

         [[16318,     0],
          [ 3106,     0]],

         [[13293,     0],
          [ 3069,     0]],

         [[17089,     0],
          [ 3055,     0]],

         [[20628,     0],
          [ 2997,     0]],

         [[20883,     0],
          [ 2960,     0]],

         [[20460,     0],
          [ 2901,     0]],

         [[20198,     0],
          [ 2889,     0]],

         [[21460,     0],
          [ 2826,     0]],

         [[23027,     0],
          [ 2857,     0]],

         [[22051,     0],
          [ 2807,     0]],

         [[22702,     0],
          [ 2786,     0]],

         [[16541,     0],
          [ 2727,     0]],

         [[24233,     0],
          [ 2722,     0]],

         [[21373,     0],
          [ 2687,     0]],

         [[25747,     0],
          [ 2639,     0]],

         [[26409,     0],
          [ 2456,     0]],

         [[24634,     0],
          [ 2541,     0]],

         [[27990,     0],
          [ 2526,     0]],

         [[28243,     0],
          [ 2494,     0]],

         [[27787,     0],
          [ 2432,     0]],

         [[25726,     0],
          [ 2384,     0]],

         [[12951,     0],
          [ 2294,     0]],

         [[ 1517,     0],
          [ 2289,     0]],

         [[26017,     0],
          [ 2263,     0]],

         [[26898,     0],
          [ 2268,     0]],

         [[ 5187,  4103],
          [ 2230,     0]],

         [[29570,     0],
          [ 2249,     0]],

         [[29075,     0],
          [ 2262,     0]],

         [[29087,     0],
          [ 2286,     0]]]], device='cuda:0'), 'mask': tensor([[[[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True,  True],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]],

         [[ True, False],
          [ True, False]]]], device='cuda:0'), 'type_ids': tensor([[[[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]],

         [[0, 0],
          [0, 0]]]], device='cuda:0')}}

In [18]:
## Testing TableembedderTorch module
from table_embedder.models.finetune_entitables_cp import TableEmbedder, TableEmbedderHypers
hypers = TableEmbedderHypers()
model = TableEmbedder(hypers).to('cuda:0')

In [21]:
loss, prob = model(table_info,indexed_headers, indexed_cells, None)

> /work/arnaik_umass_edu/tabbie_dev/table_embedder/models/finetune_entitables_cp.py(142)get_tabemb()
    140             if i == 1:
    141                 pdb.set_trace()
--> 142                 bert_data = TableUtil.add_cls_tokens(bert_header, bert_data, cls_row, cls_col, bs, n_rows, n_cols)
    143                 bert_data += row_pos_embs.expand((bs, n_cols, n_rows + 1, 768)).permute(0, 2, 1, 3).expand_as(bert_data)
    144                 bert_data += col_pos_embs.expand((bs, n_rows + 1, n_cols, 768)).expand_as(bert_data)



ipdb>  n


> /work/arnaik_umass_edu/tabbie_dev/table_embedder/models/finetune_entitables_cp.py(143)get_tabemb()
    141                 pdb.set_trace()
    142                 bert_data = TableUtil.add_cls_tokens(bert_header, bert_data, cls_row, cls_col, bs, n_rows, n_cols)
--> 143                 bert_data += row_pos_embs.expand((bs, n_cols, n_rows + 1, 768)).permute(0, 2, 1, 3).expand_as(bert_data)
    144                 bert_data += col_pos_embs.expand((bs, n_rows + 1, n_cols, 768)).expand_as(bert_data)
    145                 # table_mask = TableUtil.add_cls_mask(table_mask, table_info, bs, n_rows, n_cols, self.device)



ipdb>  row_pos_ids.shape


torch.Size([31])


...

58


ipdb>  self.hypers.num_max_row_pos


31


ipdb>  exit()


BdbQuit: 

In [ ]:
prob

In [4]:
# Load the torch Model
import torch
model_namedparam = torch.load('/work/arnaik_umass_edu/model_named.pt')
model_param = torch.load('/work/arnaik_umass_edu/model.pt')

In [1]:
#model_param

In [6]:
type(model_namedparam)

dict

In [2]:
import numpy as np

In [7]:
p = np.load('/work/arnaik_umass_edu/tabbie/data/clsrow.npy')

(768,)

In [5]:
table_info = [{'table_id': '1438042981460.12__CC-MAIN-20150728002301-00003-ip-10-236-191-2.ec2.internal__1387'
               , 'num_rows': 4
               , 'num_cols': 7
               , 'header': ['size', 'us', 'type', 'china', 'gaa', 'eu', 'years']
               , 'cell_labels': None
               , 'col_labels': [2, 4, 6]
               , 'table_labels': None
               , 'table_data_raw': [['xs', '6/7', '9.125', '235', '23', '36-37', '4']
                                    , ['s', '7/8', '9.5', '245', '24', '38-39', '5']
                                    , ['m', '8/9', '9.875', '255', '25', '39-40', '6']
                                    , ['l', '9/10', '10.375', '265', '26', '40-41', '7']]
               , 'table': [['size', 'us', 'type', 'china', 'gaa', 'eu', 'years']
                           , ['xs', '6/7', '9.125', '235', '23', '36-37', '4']
                           , ['s', '7/8', '9.5', '245', '24', '38-39', '5']
                           , ['m', '8/9', '9.875', '255', '25', '39-40', '6']
                           , ['l', '9/10', '10.375', '265', '26', '40-41', '7']]}]

In [6]:
indexed_headers = {'bert': {'token_ids': tensor([[[  101,  2946,   102],
         [  101,  2149,   102],
         [  101,  2828,   102],
         [  101,  2859,   102],
         [  101, 19930,   102],
         [  101,  7327,   102],
         [  101,  2086,   102]]], device='cuda:0')
, 'mask': tensor([[[True, True, True],
         [True, True, True],
         [True, True, True],
         [True, True, True],
         [True, True, True],
         [True, True, True],
         [True, True, True]]], device='cuda:0')
, 'type_ids': tensor([[[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]]], device='cuda:0')}}

In [7]:
indexed_cells = {'bert': {'token_ids': tensor([[[[  101,  1060,  2015,   102,     0],
          [  101,  1020,  1013,  1021,   102],
          [  101,  1023,  1012,  8732,   102],
          [  101, 17825,   102,     0,     0],
          [  101,  2603,   102,     0,     0],
          [  101,  4029,  1011,  4261,   102],
          [  101,  1018,   102,     0,     0]],

         [[  101,  1055,   102,     0,     0],
          [  101,  1021,  1013,  1022,   102],
          [  101,  1023,  1012,  1019,   102],
          [  101, 21005,   102,     0,     0],
          [  101,  2484,   102,     0,     0],
          [  101,  4229,  1011,  4464,   102],
          [  101,  1019,   102,     0,     0]],

         [[  101,  1049,   102,     0,     0],
          [  101,  1022,  1013,  1023,   102],
          [  101,  1023,  1012, 27658,   102],
          [  101, 20637,   102,     0,     0],
          [  101,  2423,   102,     0,     0],
          [  101,  4464,  1011,  2871,   102],
          [  101,  1020,   102,     0,     0]],

         [[  101,  1048,   102,     0,     0],
          [  101,  1023,  1013,  2184,   102],
          [  101,  2184,  1012, 18034,   102],
          [  101, 20549,   102,     0,     0],
          [  101,  2656,   102,     0,     0],
          [  101,  2871,  1011,  4601,   102],
          [  101,  1021,   102,     0,     0]]]], device='cuda:0')
, 'mask': tensor([[[[ True,  True,  True,  True, False],
          [ True,  True,  True,  True,  True],
          [ True,  True,  True,  True,  True],
          [ True,  True,  True, False, False],
          [ True,  True,  True, False, False],
          [ True,  True,  True,  True,  True],
          [ True,  True,  True, False, False]],

         [[ True,  True,  True, False, False],
          [ True,  True,  True,  True,  True],
          [ True,  True,  True,  True,  True],
          [ True,  True,  True, False, False],
          [ True,  True,  True, False, False],
          [ True,  True,  True,  True,  True],
          [ True,  True,  True, False, False]],

         [[ True,  True,  True, False, False],
          [ True,  True,  True,  True,  True],
          [ True,  True,  True,  True,  True],
          [ True,  True,  True, False, False],
          [ True,  True,  True, False, False],
          [ True,  True,  True,  True,  True],
          [ True,  True,  True, False, False]],

         [[ True,  True,  True, False, False],
          [ True,  True,  True,  True,  True],
          [ True,  True,  True,  True,  True],
          [ True,  True,  True, False, False],
          [ True,  True,  True, False, False],
          [ True,  True,  True,  True,  True],
          [ True,  True,  True, False, False]]]], device='cuda:0')
, 'type_ids': tensor([[[[0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0]],

         [[0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0]],

         [[0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0]],

         [[0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0]]]], device='cuda:0')}}